In [ ]:
import ast
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt


In [ ]:
# Define the path
path = '### Your file path ###'
file = "###{hemorrhage_type coordinates}.csv"
file_path = os.path.join(path, file)
df = pd.read_csv(file_path)

# df.head()

In [ ]:
# defined image_width and image_height
image_width, image_height = 512, 512
output_path = '### Output images path'


for i in range(len(df)):
    origin_name = df.loc[i, 'Origin']
    correct_coordinate = df.loc[i, 'Correct Label']
    majority_coordinate = df.loc[i, 'Majority Label']

    # If correct_coordinate is NaN，Use majority_coordinate
    if pd.isna(correct_coordinate):
        coordinates = ast.literal_eval(majority_coordinate)

    else:
        coordinates = ast.literal_eval(correct_coordinate)

    mask = np.zeros((image_height, image_width), dtype=np.uint8)

    # Extract majority_coordinate to applicable format
    if isinstance(coordinates[0], dict):
        coordinates = [coordinates]  # If it's dict, convert it to a List contains each dict.

    for contour in coordinates:
        points = np.array([(int(p['x'] * image_width), int(p['y'] * image_height)) for p in contour], dtype=np.int32)

        try:
            cv2.fillPoly(mask, [points], 255)
        except Exception as e:
            print(f"Error processing {origin_name}: {e}")

    # Save the mask image
    mask_filename = os.path.splitext(origin_name)[0] + '_mask.jpg'
    mask_filepath = os.path.join(output_path, mask_filename)
    cv2.imwrite(mask_filepath, mask)